In [1]:
%load_ext autoreload
%autoreload 2
import os
project_name = "PaddleSeg"
os.chdir(os.path.join(os.getcwd().split(project_name, 1)[0], project_name))

In [3]:
from utilities.dataset_manager import DatasetManager
from utilities.runner import Runner
root_directory = 'Z:\datasets\cracks\КЦТ_Фото_Шахты'
dm = DatasetManager()
runner = Runner()

In [5]:
extracted_subpaths = dm.copy_relevant_datasets(root_directory)

In [ ]:
for i, (path, subpath) in enumerate(extracted_subpaths):
  image_path = os.path.join('Z:\datasets\cracks\\tests', f'{i}')
  dpath = os.path.join('P:\PaddleSeg\outputs\hrsegnetb48\\result', f'{i}')
  runner.run_model(image_path, dpath)

In [6]:
image_path = os.path.join('Z:\datasets\cracks\\tests', f'selected')
dpath = os.path.join('P:\PaddleSeg\outputs\hrsegnetb48\\result', f'selected')
runner.run_model(image_path, dpath)

2023-12-01 12:09:37 [WARNING]	Add the `num_classes` in train_dataset and val_dataset config to model config. We suggest you manually set `num_classes` in model config.
2023-12-01 12:09:37 [INFO]	
------------Environment Information-------------
platform: Windows-10-10.0.22621-SP0
Python: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
Paddle compiled with cuda: True
NVCC: Build cuda_12.0.r12.0/compiler.31968024_0
cudnn: 8.9
GPUs used: 1
CUDA_VISIBLE_DEVICES: None
GPU: ['GPU 0: NVIDIA GeForce']
PaddleSeg: 0.0.0.dev0
PaddlePaddle: 2.5.2
OpenCV: 4.5.5
------------------------------------------------
2023-12-01 12:09:37 [INFO]	
---------------Config Information---------------
batch_size: 32
iters: 100000
train_dataset:
  dataset_root: data/crackseg9k
  mode: train
  num_classes: 2
  train_path: data/crackseg9k/train.txt
  transforms:
  - max_scale_factor: 2.0
    min_scale_factor: 0.5
    scale_step_size: 0.25
    type: ResizeStepScaling
  - crop_size:
  

11/11 [==============================] - 4s 365ms/step


2023-12-01 12:09:43 [INFO]	Predicted images are saved in P:\PaddleSeg\outputs\hrsegnetb48\result\selected\added_prediction and P:\PaddleSeg\outputs\hrsegnetb48\result\selected\pseudo_color_prediction .
